# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy.stats import linregress
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)
# cities


619

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [8]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "Imperial"

# Build query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="
# pprint(query_url)

limit = 0
lat = []
lng = []
temp = []
date = []
humd = []
wind = []
cloud = []
city = []
country = []

# Make a request for each of the indices
for x in range(len(cities)):
    print(f"Making request number: {x} for ID: {cities[x]}")
    response = requests.get(query_url + cities[x]).json()

#     pprint(response)

    try:
        city.append(response['name'])
        country.append(response['sys']['country'])
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        date.append(response['dt'])
        temp.append(response['main']['temp_max'])
        humd.append(response['main']['humidity'])
        wind.append(response['wind']['speed'])
        cloud.append(response['clouds']['all'])
    except:
        print("value error pass")
       
    limit += 1
    if limit == 50: break
    time.sleep(1)  
    
    


Making request number: 0 for ID: port alfred
Making request number: 1 for ID: garden city
Making request number: 2 for ID: jamestown
Making request number: 3 for ID: atuona
Making request number: 4 for ID: bubaque
Making request number: 5 for ID: bredasdorp
Making request number: 6 for ID: hithadhoo
Making request number: 7 for ID: chuy
Making request number: 8 for ID: emerald
Making request number: 9 for ID: bethel
Making request number: 10 for ID: barrow
Making request number: 11 for ID: anadyr
Making request number: 12 for ID: cidreira
Making request number: 13 for ID: gat
Making request number: 14 for ID: punta arenas
Making request number: 15 for ID: kaifeng
Making request number: 16 for ID: albany
Making request number: 17 for ID: souillac
Making request number: 18 for ID: yellowknife
Making request number: 19 for ID: ostersund
Making request number: 20 for ID: avarua
Making request number: 21 for ID: bandarbeyla
Making request number: 22 for ID: kargat
Making request number: 23 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [10]:
# Export the city data into a .csv

# create a data frame from cities, lat, and temp
weather_df = dict({
    "city": city,
    "country": country,
    "date": date,
    "lat": lat,
    "lng": lng,
    "temp": temp,
    "humd": humd,
    "wind": wind,
    "cloud": cloud
})

weather_data = pd.DataFrame.from_dict(weather_df)
# weather_data = weather_data.transpose()
weather_data.to_csv("../output_data/Weather_Data.csv", index=False)
# Convert Unix Time Stamp to regular date
tmpdate = datetime.datetime.fromtimestamp(int(weather_data["date"][0])).strftime('%Y-%m-%d')
weather_data

,city,country,date,lat,lng,temp,humd,wind,cloud
0,Port Alfred,ZA,1613178619,-33.5906,26.8910,75.00,95,5.01,73
1,Garden City,US,1613178750,42.3256,-83.3310,24.80,79,4.61,90
2,Jamestown,US,1613178482,42.0970,-79.2353,15.80,73,5.75,75
3,Atuona,PF,1613178752,-9.8000,-139.0333,79.14,78,14.56,4
4,Bubaque,GW,1613178753,11.2833,-15.8333,78.40,63,11.25,0
5,Bredasdorp,ZA,1613178622,-34.5322,20.0403,69.80,83,2.30,15
6,Hithadhoo,MV,1613178756,-0.6000,73.0833,82.24,73,11.59,95
7,Chui,UY,1613178757,-33.6971,-53.4616,70.74,92,8.12,100
8,Emerald,AU,1613178668,-23.5333,148.1667,84.20,45,9.22,2
9,Bethel,US,1613178668,41.3712,-73.4140,28.40,30,3.44,90


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.

humidityover = weather_data.loc[weather_data['humd'] > 100]
print(humidityover)


# index = weather_data.index
# humidityover = weather_data["humd"] > 100
# humdindices = index[humidityover] get only rows with > 100
# indices_list = humdindices.tolist()
# print(indices_list)


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
# scatter plot
plt.scatter(weather_data["lat"],weather_data["temp"], marker="o", alpha = 0.75)

# labels and ttile
plt.title("City Latitude vs. Temperature (" + tmpDate + ")")
plt.xlabel("Latitude")
plt.ylabel("Temperature (F)")
plt.grid(True)

# Save the figure
plt.savefig("latandtemp.png")

# Show plot
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
# scatter plot
plt.scatter(weather_data["lat"],weather_data["humd"], marker="o", alpha = 0.75)

# labels and title
plt.title("City Latitude vs. Humidity (" + tmpDate + ")")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.grid(True)

# Save the figure
plt.savefig("latandhumd.png")

# Show plot
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
# scatter plot
plt.scatter(weather_data["lat"],weather_data["cloud"], marker="o", alpha = 0.75)

# labels and title
plt.title("City Latitude vs. Cloudiness (" + tmpDate + ")")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid(True)

# Save the figure
plt.savefig("latandcloud.png")

# Show plot
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
# scatter plot
plt.scatter(weather_data["lat"],weather_data["wind"], marker="o", alpha = 0.75)

# labels and title
plt.title("City Latitude vs. Wind Speed (" + tmpDate + ")")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.grid(True)

# Save the figure
plt.savefig("latandwind.png")

# Show plot
plt.show()

## Linear Regression

In [ ]:
# northern hemisphre is north of equater, while southern hemishere is south of equator. sciencing.com(2018)
northern = weather_data.loc[weather_data["lat"] >= 0.0]
northern.reset_index(inplace=True)

southern = weather_data.loc[weather_data["lat"] < 0.0]
southern.reset_index(inplace=True)



####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# assign x and y values
x_values = northern["lat"]
y_values = northern["temp"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.ylabel('Maximum Temperature')
plt.xlabel('Latitude')
plt.title("Northern Hemisphere - Max Temp vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
## assign x and y values
x_values = southern["lat"]
y_values = southern["temp"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.ylabel('Maximum Temperature')
plt.xlabel('Latitude')
plt.title("Southern Hemisphere - Max Temp vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# assign x and y values
x_values = northern["lat"]
y_values = northern["humd"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.ylabel('Humidity (%)')
plt.xlabel('Latitude')
plt.title("Northern Hemisphere - Humidity (%) vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# assign x and y values
x_values = southern["lat"]
y_values = southern["humd"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot

plt.ylabel('Humidity')
plt.xlabel('Latitude')
plt.title("Southern Hemisphere - Maximum Humidity (%) vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# assign x and y values
x_values = northern["lat"]
y_values = northern["cloud"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.ylabel('Cloudiness')
plt.xlabel('Latitude')
plt.title("Northern Hemisphere - Cloudiness vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# assign x and y values
x_values = southern["lat"]
y_values = southern["cloud"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.ylabel('Cloudiness')
plt.xlabel('Latitude')
plt.title("Southern Hemisphere - Cloudiness vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# assign x and y values
x_values = northern["lat"]
y_values = northern["wind"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.ylabel('Wind Speed (mph)')
plt.xlabel('Latitude')
plt.title("Northern Hemisphere - Wind Speed (mph) vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# assign x and y values
x_values = southern["lat"]
y_values = southern["wind"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.ylabel('Wind Speed (mph)')
plt.xlabel('Latitude')
plt.title("Southern Hemisphere - Wind Speed (mph) vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()